In [1]:
# 导入必要的库
# 从diffusers库导入Stable Diffusion相关的管道和模型
from diffusers import StableDiffusionPipeline, UNet2DConditionModel, StableDiffusionXLPipeline
import torch

# 禁用梯度计算，提高推理速度
torch.set_grad_enabled(False)

The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.


In [ ]:
# 加载经过DPO(Direct Preference Optimization)训练的UNet模型
# DPO是一种直接偏好优化方法，可以提高生成图像的质量
dpo_unet = UNet2DConditionModel.from_pretrained(
                            #  'mhdang/dpo-sd1.5-text2image-v1',  # SD1.5版本的DPO模型
                            'mhdang/dpo-sdxl-text2image-v1',      # SDXL版本的DPO模型
                            # 或者使用本地检查点目录 (*/checkpoint-n/)
                            subfolder='unet',        # 指定UNet子文件夹
                            torch_dtype=torch.float16  # 使用半精度浮点数节省显存
).to('cuda')  # 将模型移动到GPU

diffusion_pytorch_model.safetensors:  55%|#####4    | 5.62G/10.3G [00:00<?, ?B/s]

In [ ]:
# 设置预训练模型名称
# 可以选择不同版本的Stable Diffusion模型
# pretrained_model_name = "CompVis/stable-diffusion-v1-4"          # SD v1.4版本
# pretrained_model_name = "runwayml/stable-diffusion-v1-5"         # SD v1.5版本
pretrained_model_name = "stabilityai/stable-diffusion-xl-base-1.0"  # SDXL基础版本

# 根据模型类型设置引导强度(guidance scale)
# SDXL使用5.0，SD1.x使用7.5作为默认值
gs = (5 if 'stable-diffusion-xl' in pretrained_model_name else 7.5)

In [ ]:
# 根据模型类型创建对应的Stable Diffusion管道
if 'stable-diffusion-xl' in pretrained_model_name:
    # 创建SDXL管道
    pipe = StableDiffusionXLPipeline.from_pretrained(
        pretrained_model_name, 
        torch_dtype=torch.float16,  # 使用半精度浮点数
        variant="fp16",             # 加载fp16版本模型
        use_safetensors=True        # 使用安全张量格式
    ).to("cuda")
else:
    # 创建标准SD管道
    pipe = StableDiffusionPipeline.from_pretrained(pretrained_model_name,
                                                   torch_dtype=torch.float16)
    pipe = pipe.to('cuda')

# 禁用内容安全检查器，避免过度敏感的内容过滤
# 注意：安全检查器有时会错误地将正常内容(如"机器人老虎")标记为不安全
pipe.safety_checker = None



In [ ]:
# 导入评分工具
# 可以选择使用 clip_utils（CLIP评分）, aes_utils（美学评分）, hps_utils（人类偏好评分）等
from utils.pickscore_utils import Selector

# 使用奖励模型自动为生成的图像评分
# PickScore是一个基于人类偏好训练的评分模型
ps_selector = Selector('cuda')

In [ ]:
# 准备对比的UNet模型列表
# 第一个是原始SDXL模型，第二个是经过DPO优化的模型
unets = [pipe.unet, dpo_unet]

# 对应的模型名称，用于显示和区分
names = ["原始 SDXL", "DPO SDXL"]

In [ ]:
def gen(prompt, seed=0, run_baseline=True):
    """
    生成图像并对比不同模型的效果
    
    参数:
        prompt (str): 文本提示词，用于指导图像生成
        seed (int): 随机种子，确保结果可复现，默认为0
        run_baseline (bool): 是否同时运行基线模型对比，默认为True
    
    返回:
        list: 生成的图像列表，如果run_baseline=True则包含两张图像（原始模型+DPO模型）
              如果run_baseline=False则只包含一张图像（仅DPO模型）
    
    功能说明:
        - 使用相同的seed确保不同模型生成的图像具有可比性
        - 支持选择性对比（可以只生成DPO模型的结果）
        - 自动显示生成的图像并返回图像对象供后续使用
    """
    ims = []  # 存储生成的图像
    generator = torch.Generator(device='cuda')  # 创建GPU上的随机数生成器
    
    # 根据run_baseline参数决定运行哪些模型：[0,1]表示原始+DPO，[1]表示仅DPO
    for unet_i in ([0, 1] if run_baseline else [1]):
        print(f"提示词: {prompt}\n随机种子: {seed}\n当前模型: {names[unet_i]}")
        
        # 切换到对应的UNet模型
        pipe.unet = unets[unet_i]
        
        # 设置随机种子，确保可复现性
        generator = generator.manual_seed(seed)
        
        # 生成图像
        im = pipe(prompt=prompt, generator=generator, guidance_scale=gs).images[0]
        
        # 显示生成的图像
        display(im)
        
        # 将图像添加到结果列表
        ims.append(im)
    
    return ims

In [ ]:
# 示例提示词列表
# 包含各种创意和艺术风格的提示词，用于测试模型的生成能力
example_prompts = [
    "A pile of sand swirling in the wind forming the shape of a dancer",
    "A giant dinosaur frozen into a glacier and recently discovered by scientists, cinematic still",
    "a smiling beautiful sorceress with long dark hair and closed eyes wearing a dark top surrounded by glowing fire sparks at night, magical light fog, deep focus+closeup, hyper-realistic, volumetric lighting, dramatic lighting, beautiful composition, intricate details, instagram, trending, photograph, film grain and noise, 8K, cinematic, post-production",
    "A purple raven flying over big sur, light fog, deep focus+closeup, hyper-realistic, volumetric lighting, dramatic lighting, beautiful composition, intricate details, instagram, trending, photograph, film grain and noise, 8K, cinematic, post-production",
    "a smiling beautiful sorceress wearing a modest high necked blue suit surrounded by swirling rainbow aurora, hyper-realistic, cinematic, post-production",
    "Anthro humanoid turtle skydiving wearing goggles, gopro footage",
    "A man in a suit surfing in a river",
    "photo of a zebra dressed suit and tie sitting at a table in a bar with a bar stools, award winning photography",
    "A typhoon in a tea cup, digital render",
    "A cute puppy leading a session of the United Nations, newspaper photography",
    "Worm eye view of rocketship",
    "Glass spheres in the desert, refraction render",
    "anthropmorphic coffee bean drinking coffee",
    "A baby kangaroo in a trenchcoat",
    "A towering hurricane of rainbow colors towering over a city, cinematic digital art",
    "A redwood tree rising up out of the ocean",
]


# 批量生成和评分流程
# 对每个提示词进行图像生成，并使用PickScore模型进行自动评分
for p in example_prompts:
    # 生成图像（可以选择保存这些图像）
    ims = gen(p)
    
    # 使用PickScore对生成的图像进行评分
    # 分数越高表示图像质量越好，越符合人类偏好
    scores = ps_selector.score(ims, p)
    print(f"评分结果: {scores}")

In [ ]:
# 加载PartiPrompts数据集
# PartiPrompts是Google发布的用于评估文本到图像生成模型的基准数据集
# 包含1600多个精心设计的提示词，涵盖各种复杂性和类别
from datasets import load_dataset

# 从Hugging Face Hub加载PartiPrompts数据集
dataset = load_dataset("nateraw/parti-prompts")

# 打印训练集中的所有提示词
# 这些提示词可以用于更全面的模型性能评估
print("PartiPrompts数据集中的提示词:")
print(dataset['train']['Prompt'])

In [ ]:
# 这是一个空的单元格，可以用于额外的实验和测试代码
# 
# 使用示例:
# images = gen("自定义提示词", seed=123)
# scores = ps_selector.score(images, "自定义提示词") 
#
# 提示：运行上面的单元格来体验DPO优化后的Stable Diffusion模型效果
